In [ ]:
# Define the dimensional model structure
class SatelliteDataModel:
    """
    Star schema for Space Situational Awareness dashboard
    """
    
    # FACT TABLE - Satellite Observations
        'pass_id': 'INT PRIMARY KEY',
        'satellite_id': 'INT FK',
        'location_id': 'INT FK', 
        'time_id': 'INT FK',
        'orbit_id': 'INT FK',
        'risk_id': 'INT FK',
        # Measures
        'distance_km': 'FLOAT',
        'max_elevation_deg': 'FLOAT',
        'pass_duration_sec': 'FLOAT',
        'relative_velocity_km_s': 'FLOAT',
        'collision_probability': 'FLOAT',
        'debris_density_index': 'FLOAT'
    }
    
    # DIMENSION - Satellites
    dim_satellite = {
        'satellite_id': 'INT PRIMARY KEY',
        'norad_id': 'VARCHAR(10)',
        'satellite_name': 'VARCHAR(100)',
        'object_type': 'VARCHAR(50)',  # PAYLOAD, DEBRIS, ROCKET_BODY
        'country_code': 'VARCHAR(5)',
        'launch_date': 'DATE',
        'status': 'VARCHAR(20)',  # ACTIVE, INACTIVE, DECAYED
        'purpose': 'VARCHAR(50)'  # COMMUNICATIONS, EARTH_OBS, NAVIGATION
    }
    
    # DIMENSION - Locations
    dim_location = {
        'location_id': 'INT PRIMARY KEY',
        'city_name': 'VARCHAR(100)',
        'country': 'VARCHAR(100)',
        'latitude': 'FLOAT',
        'longitude': 'FLOAT',
        'timezone': 'VARCHAR(50)',
        'population': 'INT',
        'tech_dependency_score': 'FLOAT'  # How dependent on satellite services
    }
    
    # DIMENSION - Time
    dim_time = {
        'time_id': 'INT PRIMARY KEY',
        'datetime': 'TIMESTAMP',
        'date': 'DATE',
        'year': 'INT',
        'month': 'INT',
        'day': 'INT',
        'hour': 'INT',
        'day_of_week': 'VARCHAR(10)',
        'is_weekend': 'BOOLEAN',
        'solar_activity_level': 'VARCHAR(10)'  # LOW, MEDIUM, HIGH
    }
    
    # DIMENSION - Orbital Characteristics
    dim_orbit = {
        'orbit_id': 'INT PRIMARY KEY',
        'altitude_km': 'FLOAT',
        'inclination_deg': 'FLOAT',
        'eccentricity': 'FLOAT',
        'mean_motion_rev_day': 'FLOAT',
        'orbital_period_min': 'FLOAT',
        'orbit_class': 'VARCHAR(10)',  # LEO, MEO, GEO, HEO
        'orbit_type': 'VARCHAR(30)'  # SUN_SYNCHRONOUS, POLAR, EQUATORIAL
    }
    
    # DIMENSION - Risk Factors
    dim_risk = {
        'risk_id': 'INT PRIMARY KEY',
        'congestion_score': 'FLOAT',  # 0-1 scale
        'debris_risk_score': 'FLOAT',  # 0-1 scale
        'crossing_risk_score': 'FLOAT',  # 0-1 scale
        'total_risk_score': 'FLOAT',  # 0-100 scale
        'risk_level': 'VARCHAR(10)',  # LOW, MEDIUM, HIGH, CRITICAL
        'kessler_probability': 'FLOAT',  # Cascade event probability
        'mitigation_priority': 'INT'  # 1-10 scale
    }

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def create_dimensional_model(raw_satellite_data, cities):
    """
    Transform raw data into star schema format
    """
    
    # Create dimension tables
    dim_satellite_df = create_satellite_dimension(raw_satellite_data)
    dim_location_df = create_location_dimension(cities)
    dim_time_df = create_time_dimension()
    dim_orbit_df = create_orbit_dimension(raw_satellite_data)
    dim_risk_df = create_risk_dimension()
    
    # Create fact table
    fact_passes_df = create_fact_table(
        raw_satellite_data, 
        dim_satellite_df,
        dim_location_df,
        dim_time_df,
        dim_orbit_df,
        dim_risk_df
    )
    
    return {
        'fact_satellite_passes': fact_passes_df,
        'dim_satellite': dim_satellite_df,
        'dim_location': dim_location_df,
        'dim_time': dim_time_df,
        'dim_orbit': dim_orbit_df,
        'dim_risk': dim_risk_df
    }

def create_satellite_dimension(satellites_df):
    """
    Create satellite dimension with surrogate keys
    """
    dim_satellite = pd.DataFrame({
        'satellite_id': range(1, len(satellites_df) + 1),
        'norad_id': satellites_df['norad_id'],
        'satellite_name': satellites_df['name'],
        'object_type': np.random.choice(['PAYLOAD', 'DEBRIS', 'ROCKET_BODY'], 
                                       len(satellites_df), p=[0.7, 0.2, 0.1]),
        'country_code': 'USA',  # Would need real data
        'launch_date': pd.date_range(start='2010-01-01', 
                                    periods=len(satellites_df), freq='W'),
        'status': np.where(satellites_df['altitude_km'] > 0, 'ACTIVE', 'DECAYED'),
        'purpose': np.random.choice(['COMMUNICATIONS', 'EARTH_OBS', 'NAVIGATION', 'SCIENTIFIC'],
                                   len(satellites_df))
    })
    return dim_satellite

def create_location_dimension(cities):
    """
    Create location dimension for cities
    """
    dim_location = pd.DataFrame([
        {
            'location_id': 1,
            'city_name': 'Philadelphia',
            'country': 'United States',
            'latitude': 39.9526,
            'longitude': -75.1652,
            'timezone': 'America/New_York',
            'population': 1584000,
            'tech_dependency_score': 0.85
        },
        {
            'location_id': 2,
            'city_name': 'Rio de Janeiro',
            'country': 'Brazil',
            'latitude': -22.9068,
            'longitude': -43.1729,
            'timezone': 'America/Sao_Paulo',
            'population': 6748000,
            'tech_dependency_score': 0.75
        }
    ])
    return dim_location

def create_orbit_dimension(satellites_df):
    """
    Create orbital characteristics dimension
    """
    dim_orbit = pd.DataFrame({
        'orbit_id': range(1, len(satellites_df) + 1),
        'altitude_km': satellites_df['altitude_km'],
        'inclination_deg': satellites_df['inclination'],
        'eccentricity': satellites_df['eccentricity'],
        'mean_motion_rev_day': satellites_df['mean_motion'],
        'orbital_period_min': 1440 / satellites_df['mean_motion'],
        'orbit_class': pd.cut(satellites_df['altitude_km'], 
                             bins=[0, 2000, 35786, 100000],
                             labels=['LEO', 'MEO', 'GEO']),
        'orbit_type': np.where(
            abs(satellites_df['inclination'] - 98) < 2, 'SUN_SYNCHRONOUS',
            np.where(satellites_df['inclination'] > 85, 'POLAR',
            np.where(satellites_df['inclination'] < 10, 'EQUATORIAL', 'INCLINED'))
        )
    })
    return dim_orbit

def create_risk_dimension():
    """
    Create pre-calculated risk scenarios
    """
    risk_scenarios = []
    for congestion in np.arange(0, 1.1, 0.2):
        for debris in np.arange(0, 1.1, 0.2):
            for crossing in np.arange(0, 1.1, 0.5):
                total = (congestion * 0.4 + debris * 0.4 + crossing * 0.2) * 100
                risk_scenarios.append({
                    'risk_id': len(risk_scenarios) + 1,
                    'congestion_score': round(congestion, 2),
                    'debris_risk_score': round(debris, 2),
                    'crossing_risk_score': round(crossing, 2),
                    'total_risk_score': round(total, 1),
                    'risk_level': 'CRITICAL' if total > 70 else 'HIGH' if total > 50 
                                 else 'MEDIUM' if total > 30 else 'LOW',
                    'kessler_probability': min(1.0, total / 100 * 1.5),
                    'mitigation_priority': min(10, int(total / 10))
                })
    
    return pd.DataFrame(risk_scenarios)

def calculate_risk_id(altitude, inclination, location_lat):
    """
    Map orbital parameters to risk dimension
    """
    # Simplified risk calculation logic
    congestion = min(1.0, (3000 - altitude) / 3000) if altitude < 2000 else 0.2
    debris = 0.8 if 700 < altitude < 900 else 0.3
    crossing = min(1.0, abs(inclination - abs(location_lat)) / 90)
    
    # Find nearest risk scenario
    # In production, this would be a lookup
    return 1  # Placeholder

In [3]:
def create_fact_table(satellites_df, dim_sat, dim_loc, dim_time, dim_orbit, dim_risk):
    """
    Generate fact records for satellite passes
    """
    fact_records = []
    pass_id = 1
    
    # Simulate passes for each city
    for loc_idx, location in dim_loc.iterrows():
        # Sample satellites visible from this location
        visible_sats = satellites_df.sample(n=min(100, len(satellites_df)))
        
        for sat_idx, sat in visible_sats.iterrows():
            # Calculate pass characteristics
            lat_diff = abs(sat['inclination'] - abs(location['latitude']))
            visibility = lat_diff < 90  # Simplified
            
            if visibility:
                fact_records.append({
                    'pass_id': pass_id,
                    'satellite_id': sat_idx + 1,
                    'location_id': location['location_id'],
                    'time_id': np.random.randint(1, 1000),
                    'orbit_id': sat_idx + 1,
                    'risk_id': calculate_risk_id(
                        sat['altitude_km'], 
                        sat['inclination'],
                        location['latitude']
                    ),
                    'distance_km': np.random.uniform(400, 2000),
                    'max_elevation_deg': np.random.uniform(10, 90),
                    'pass_duration_sec': np.random.uniform(60, 600),
                    'relative_velocity_km_s': 7.5 - (sat['altitude_km'] / 1000),
                    'collision_probability': np.random.exponential(0.00001),
                    'debris_density_index': np.random.exponential(10)
                })
                pass_id += 1
    
    return pd.DataFrame(fact_records)

In [14]:
import pandas as pd
import numpy as np  # ADD THIS LINE
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# First, create sample data for visualization
def generate_sample_fact_table():
    """Generate sample fact table data for visualization"""
    
    # Create sample fact table
    np.random.seed(42)
    n_records = 500
    
    fact_table = pd.DataFrame({
        'pass_id': range(1, n_records + 1),
        'satellite_id': np.random.randint(1, 101, n_records),
        'location_id': np.random.choice([1, 2], n_records),  # Philadelphia or Rio
        'time_id': np.random.randint(1, 365, n_records),
        'orbit_id': np.random.randint(1, 101, n_records),
        'risk_id': np.random.randint(1, 50, n_records),
        'distance_km': np.random.uniform(400, 2000, n_records),
        'max_elevation_deg': np.random.uniform(10, 90, n_records),
        'debris_density_index': np.random.exponential(10, n_records),
        'collision_probability': np.random.exponential(0.00001, n_records),
    })
    
    return fact_table

# Generate the data
fact_df = generate_sample_fact_table()

# Create visualizations
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Fact Table Record Distribution',
        'Satellite Passes by Location', 
        'Risk Distribution Across Passes',
        'Debris Density by Distance',
        'Star Schema Relationships',
        'Fact Table Sample Records'
    ),
    specs=[
        [{'type': 'histogram'}, {'type': 'bar'}, {'type': 'box'}],
        [{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'table'}]
    ]
)

# 1. Record distribution over time
fig.add_trace(
    go.Histogram(x=fact_df['time_id'], nbinsx=30, 
                 marker_color='blue', name='Temporal Distribution'),
    row=1, col=1
)

# 2. Passes by location
location_counts = fact_df['location_id'].value_counts()
fig.add_trace(
    go.Bar(x=['Philadelphia', 'Rio de Janeiro'], 
           y=[location_counts.get(1, 0), location_counts.get(2, 0)],
           marker_color=['green', 'orange'],
           text=[location_counts.get(1, 0), location_counts.get(2, 0)],
           textposition='outside'),
    row=1, col=2
)

# 3. Risk distribution
risk_categories = pd.cut(fact_df['risk_id'], bins=[0, 15, 30, 50], 
                         labels=['Low', 'Medium', 'High'])
fig.add_trace(
    go.Box(y=fact_df['distance_km'], x=risk_categories, 
           marker_color='red', name='Risk vs Distance'),
    row=1, col=3
)

# 4. Debris density scatter
fig.add_trace(
    go.Scatter(x=fact_df['distance_km'], y=fact_df['debris_density_index'],
               mode='markers', marker=dict(
                   size=5,
                   color=fact_df['risk_id'],
                   colorscale='Viridis',
                   showscale=True,
                   colorbar=dict(title="Risk ID", x=0.63, len=0.4)
               )),
    row=2, col=1
)

# 5. Star schema visualization using network position
star_x = [0, -1, -1, 0, 1, 1]  # Fact table at center
star_y = [0, 1, -1, -1.5, -1, 1]
star_labels = ['FACT: Passes', 'DIM: Satellite', 'DIM: Location', 
               'DIM: Time', 'DIM: Orbit', 'DIM: Risk']

fig.add_trace(
    go.Scatter(x=star_x, y=star_y, mode='markers+text',
               marker=dict(size=[40, 25, 25, 25, 25, 25],
                          color=['red', 'blue', 'blue', 'blue', 'blue', 'blue']),
               text=star_labels,
               textposition='top center'),
    row=2, col=2
)

# Add edges to show relationships
for i in range(1, 6):
    fig.add_trace(
        go.Scatter(x=[0, star_x[i]], y=[0, star_y[i]],
                   mode='lines', line=dict(color='gray', width=1),
                   showlegend=False),
        row=2, col=2
    )

# 6. Sample fact table records
sample_data = fact_df.head(5)[['pass_id', 'satellite_id', 'location_id', 
                                'risk_id', 'distance_km', 'debris_density_index']]
fig.add_trace(
    go.Table(
        header=dict(values=list(sample_data.columns),
                   fill_color='lightgray',
                   align='left'),
        cells=dict(values=[sample_data[col].round(2) for col in sample_data.columns],
                  fill_color='white',
                  align='left')),
    row=2, col=3
)

# Update layout
fig.update_layout(
    height=700,
    showlegend=False,
    title_text="Space Situational Awareness - Fact Table Analysis"
)

fig.update_xaxes(title_text="Time ID (Day of Year)", row=1, col=1)
fig.update_xaxes(title_text="City", row=1, col=2)
fig.update_xaxes(title_text="Risk Category", row=1, col=3)
fig.update_xaxes(title_text="Distance (km)", row=2, col=1)
fig.update_xaxes(showticklabels=False, showgrid=False, row=2, col=2)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Number of Passes", row=1, col=2)
fig.update_yaxes(title_text="Distance (km)", row=1, col=3)
fig.update_yaxes(title_text="Debris Density Index", row=2, col=1)
fig.update_yaxes(showticklabels=False, showgrid=False, row=2, col=2)

fig.show()

# Create a detailed analysis of fact table metrics
print("Fact Table Analytics Summary")
print("="*50)
print(f"Total Records: {len(fact_df)}")
print(f"Unique Satellites: {fact_df['satellite_id'].nunique()}")
print(f"Average Distance: {fact_df['distance_km'].mean():.1f} km")
print(f"High Risk Passes (risk_id > 30): {len(fact_df[fact_df['risk_id'] > 30])}")
print(f"Average Debris Density: {fact_df['debris_density_index'].mean():.2f}")
print(f"\nPasses by Location:")
print(f"  Philadelphia: {len(fact_df[fact_df['location_id'] == 1])}")
print(f"  Rio de Janeiro: {len(fact_df[fact_df['location_id'] == 2])}")

Fact Table Analytics Summary
Total Records: 500
Unique Satellites: 99
Average Distance: 1190.6 km
High Risk Passes (risk_id > 30): 192
Average Debris Density: 9.92

Passes by Location:
  Philadelphia: 261
  Rio de Janeiro: 239


In [15]:
# Create aggregated views for dashboard
def create_aggregated_views(fact_df):
    """Create pre-aggregated views for dashboard performance"""
    
    # Daily risk trends
    daily_risk = fact_df.groupby(['time_id', 'location_id']).agg({
        'risk_id': 'mean',
        'debris_density_index': 'mean',
        'pass_id': 'count'
    }).reset_index()
    daily_risk.columns = ['time_id', 'location_id', 'avg_risk', 'avg_debris', 'pass_count']
    
    # Satellite risk profile
    satellite_risk = fact_df.groupby('satellite_id').agg({
        'risk_id': 'mean',
        'distance_km': 'mean',
        'collision_probability': 'max'
    }).reset_index()
    
    # Location comparison
    location_stats = fact_df.groupby('location_id').agg({
        'risk_id': ['mean', 'std', 'max'],
        'debris_density_index': ['mean', 'max'],
        'pass_id': 'count'
    }).reset_index()
    
    return daily_risk, satellite_risk, location_stats

daily_risk, satellite_risk, location_stats = create_aggregated_views(fact_df)

# Visualize aggregated metrics
fig2 = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Daily Risk Trends by City',
        'Satellite Risk Profiles',
        'Location Risk Comparison',
        'Collision Probability Distribution'
    )
)

# Plot 1: Time series of risk
philly_risk = daily_risk[daily_risk['location_id'] == 1]
rio_risk = daily_risk[daily_risk['location_id'] == 2]

fig2.add_trace(
    go.Scatter(x=philly_risk['time_id'], y=philly_risk['avg_risk'],
               mode='lines', name='Philadelphia', line=dict(color='blue')),
    row=1, col=1
)
fig2.add_trace(
    go.Scatter(x=rio_risk['time_id'], y=rio_risk['avg_risk'],
               mode='lines', name='Rio de Janeiro', line=dict(color='green')),
    row=1, col=1
)

# Plot 2: Satellite risk bubble chart
fig2.add_trace(
    go.Scatter(x=satellite_risk['distance_km'], 
               y=satellite_risk['risk_id'],
               mode='markers',
               marker=dict(
                   size=satellite_risk['collision_probability']*1e6,
                   color=satellite_risk['risk_id'],
                   colorscale='RdYlGn_r',
                   showscale=True
               )),
    row=1, col=2
)

# Plot 3: Location comparison
locations = ['Philadelphia', 'Rio de Janeiro']
metrics = ['Avg Risk', 'Max Risk', 'Avg Debris']
philly_vals = [
    fact_df[fact_df['location_id']==1]['risk_id'].mean(),
    fact_df[fact_df['location_id']==1]['risk_id'].max(),
    fact_df[fact_df['location_id']==1]['debris_density_index'].mean()
]
rio_vals = [
    fact_df[fact_df['location_id']==2]['risk_id'].mean(),
    fact_df[fact_df['location_id']==2]['risk_id'].max(),
    fact_df[fact_df['location_id']==2]['debris_density_index'].mean()
]

fig2.add_trace(
    go.Bar(name='Philadelphia', x=metrics, y=philly_vals, marker_color='blue'),
    row=2, col=1
)
fig2.add_trace(
    go.Bar(name='Rio de Janeiro', x=metrics, y=rio_vals, marker_color='green'),
    row=2, col=1
)

# Plot 4: Collision probability histogram
fig2.add_trace(
    go.Histogram(x=fact_df['collision_probability']*1e6, nbinsx=30,
                 marker_color='red'),
    row=2, col=2
)

fig2.update_layout(height=700, showlegend=True, 
                   title_text="Fact Table Aggregated Analytics")
fig2.update_xaxes(title_text="Day of Year", row=1, col=1)
fig2.update_xaxes(title_text="Average Distance (km)", row=1, col=2)
fig2.update_xaxes(title_text="Metric", row=2, col=1)
fig2.update_xaxes(title_text="Collision Probability (×10⁻⁶)", row=2, col=2)

fig2.show()

# Print dimensional cardinality
print("\nDimensional Cardinality Analysis")
print("="*50)
print(f"Satellite Dimension: {fact_df['satellite_id'].nunique()} unique satellites")
print(f"Location Dimension: {fact_df['location_id'].nunique()} cities")
print(f"Time Dimension: {fact_df['time_id'].nunique()} unique time periods")
print(f"Orbit Dimension: {fact_df['orbit_id'].nunique()} unique orbital profiles")
print(f"Risk Dimension: {fact_df['risk_id'].nunique()} risk scenarios")
print(f"\nFact Density: {len(fact_df) / (fact_df['satellite_id'].nunique() * fact_df['location_id'].nunique()):.1f} passes per satellite-location pair")


Dimensional Cardinality Analysis
Satellite Dimension: 99 unique satellites
Location Dimension: 2 cities
Time Dimension: 269 unique time periods
Orbit Dimension: 100 unique orbital profiles
Risk Dimension: 49 risk scenarios

Fact Density: 2.5 passes per satellite-location pair


In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Generate the fact table data
np.random.seed(42)
n_records = 500

fact_table = pd.DataFrame({
    'pass_id': range(1, n_records + 1),
    'satellite_id': np.random.randint(1, 101, n_records),
    'location_id': np.random.choice([1, 2], n_records),
    'time_id': np.random.randint(1, 365, n_records),
    'orbit_id': np.random.randint(1, 101, n_records),
    'risk_id': np.random.randint(1, 50, n_records),
    'distance_km': np.round(np.random.uniform(400, 2000, n_records), 2),
    'max_elevation_deg': np.round(np.random.uniform(10, 90, n_records), 2),
    'debris_density_index': np.round(np.random.exponential(10, n_records), 3),
    'collision_probability': np.round(np.random.exponential(0.00001, n_records), 8),
})

# Create dimension tables
dim_location = pd.DataFrame({
    'location_id': [1, 2],
    'city_name': ['Philadelphia', 'Rio de Janeiro'],
    'country': ['United States', 'Brazil'],
    'latitude': [39.9526, -22.9068],
    'longitude': [-75.1652, -43.1729]
})

dim_satellite = pd.DataFrame({
    'satellite_id': range(1, 101),
    'satellite_name': [f'SAT-{i:04d}' for i in range(1, 101)],
    'object_type': np.random.choice(['PAYLOAD', 'DEBRIS', 'ROCKET_BODY'], 100, p=[0.7, 0.2, 0.1]),
    'status': np.random.choice(['ACTIVE', 'INACTIVE'], 100, p=[0.8, 0.2])
})

dim_risk = pd.DataFrame({
    'risk_id': range(1, 51),
    'risk_level': pd.cut(range(1, 51), bins=[0, 15, 30, 50], labels=['LOW', 'MEDIUM', 'HIGH']),
    'congestion_score': np.round(np.random.uniform(0, 1, 50), 3),
    'debris_risk_score': np.round(np.random.uniform(0, 1, 50), 3)
})

# Create Excel file with multiple sheets
excel_filename = 'ssa_fact_table.xlsx'

with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    # Write fact table
    fact_table.to_excel(writer, sheet_name='FACT_Satellite_Passes', index=False)
    
    # Write dimension tables
    dim_location.to_excel(writer, sheet_name='DIM_Location', index=False)
    dim_satellite.to_excel(writer, sheet_name='DIM_Satellite', index=False)
    dim_risk.to_excel(writer, sheet_name='DIM_Risk', index=False)
    
    # Create summary statistics sheet
    summary_stats = pd.DataFrame({
        'Metric': [
            'Total Passes',
            'Unique Satellites',
            'Average Distance (km)',
            'Max Elevation (deg)',
            'High Risk Passes',
            'Philadelphia Passes',
            'Rio de Janeiro Passes',
            'Average Debris Density'
        ],
        'Value': [
            len(fact_table),
            fact_table['satellite_id'].nunique(),
            round(fact_table['distance_km'].mean(), 2),
            round(fact_table['max_elevation_deg'].max(), 2),
            len(fact_table[fact_table['risk_id'] > 30]),
            len(fact_table[fact_table['location_id'] == 1]),
            len(fact_table[fact_table['location_id'] == 2]),
            round(fact_table['debris_density_index'].mean(), 3)
        ]
    })
    summary_stats.to_excel(writer, sheet_name='Summary_Statistics', index=False)
    
    # Format the Excel file
    for sheet_name in writer.sheets:
        worksheet = writer.sheets[sheet_name]
        
        # Adjust column widths
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 30)
            worksheet.column_dimensions[column_letter].width = adjusted_width

print(f"Excel file '{excel_filename}' created successfully!")
print(f"Contains {len(fact_table)} fact records across 5 sheets")

Excel file 'ssa_fact_table.xlsx' created successfully!
Contains 500 fact records across 5 sheets


In [19]:
# Create formatted Excel with conditional formatting
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.formatting.rule import ColorScaleRule

# First create the basic Excel
fact_table.to_excel('formatted_fact_table.xlsx', index=False)

# Load and format
wb = load_workbook('formatted_fact_table.xlsx')
ws = wb.active

# Add header formatting
header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
header_font = Font(color='FFFFFF', bold=True)

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal='center')

# Add conditional formatting to risk_id column
risk_col = 'F'  # Column F is risk_id
ws.conditional_formatting.add(f'{risk_col}2:{risk_col}{len(fact_table)+1}',
                              ColorScaleRule(start_type='min', start_color='90EE90',
                                           mid_type='percentile', mid_value=50, mid_color='FFFF00',
                                           end_type='max', end_color='FF0000'))

# Save formatted file
wb.save('formatted_fact_table.xlsx')
print("Formatted Excel file created with color-coded risk levels")

Formatted Excel file created with color-coded risk levels
